# Run this cell to set up paths, imports, and W&B logging

In [1]:
%cd ..

import os
from PIL import Image

import wandb
import numpy as np
import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from src.model import SimpleCNN
from src.training import train, validate

/home/mytkom/Documents/iml


# 1. Initialize W&B project

In [2]:
RUN_NAME="SimpleCNN_1"

wandb.init(name="", project="iml", config={
    "learning_rate": 0.001,
    "epochs": 80,
    "batch_size": 32,
    "image_size": (32, 32)
})
config = wandb.config

# Configure directories
TRAIN_DIR = "./datasets/train/"
VAL_DIR = "./datasets/val/"
TEST_DIR = "./datasets/test/"

# Speakers of such labels should be admitted to enter
VALID_ACCESS_LABELS = {
    "f1", "f7", "f8", "m3", "m6", "m8"
}

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mytkom (mytkom-warsaw-university-of-technology). Use `wandb login --relogin` to force relogin


# 2. Define a custom dataset class to load spectrograms from files

In [3]:
class SpectrogramDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        speaker_id = img_path.split('/')[-1].split('_')[0]
        label = int(speaker_id in VALID_ACCESS_LABELS)
        image = Image.open(img_path).convert("L")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# 3. Set up data transformations and data loaders

In [4]:
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
])

train_dataset = SpectrogramDataset(TRAIN_DIR, transform=transform)
val_dataset = SpectrogramDataset(VAL_DIR, transform=transform)
test_dataset = SpectrogramDataset(TEST_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)



# 4. Initialize model, optimizer, and set device

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)


# 5. Training and validation loop

In [6]:

for epoch in range(config.epochs):
    print(f"Epoch {epoch+1}/{config.epochs}")
    
    train_loss = train(model, train_loader, optimizer, epoch)
    val_f1 = validate(model, val_loader)
    wandb.log({"Train Loss": train_loss, "Validation F1": val_f1, "Epoch": epoch+1})
    print("Train Loss: ", train_loss, ", Validation F1", val_f1)


Epoch 1/3
Train Loss:  0.6292004585266113 , Epoch:  0
{'Validation F1': np.float64(0.382804995196926)}
Epoch 2/3
Train Loss:  0.40982311964035034 , Epoch:  1
{'Validation F1': np.float64(0.7060663517558026)}
Epoch 3/3
Train Loss:  0.4516843259334564 , Epoch:  2
{'Validation F1': np.float64(0.8228201697494613)}



# 6. Save the model

In [7]:

model_path = "./models/simple_cnn.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()

print("Training complete and model saved!")


Epoch,▁▃▃▆▆█
Train Loss,█▁▂
Validation F1,▁▁▆▆██
Epoch,3
Validation F1,0.82282


Training complete and model saved!
